In [1]:
##连接Spark
import taiji_ide #用于选择spark的版本
taiji_ide.set_spark(version="3.1.2",gaia_id="3527") #导入spark环境，选择spark版本
import os
from pyspark.sql import *
os.environ['GROUP_ID'] = 'g_wxg_wxa_wxa_offline_datamining'
os.environ['GAIA_ID'] = '3527'
session = SparkSession.builder.config('spark.driver.memory', '32g').config('spark.executor.cores', 8).config('spark.executor.memory', '32g').getOrCreate()

Driver Log Write In: /tmp/log/spark-bf987129-4f5a-4bc4-9027-2273ce457011.log
TDWHelper Tracking Url : http://tdwhelper.oa.com/resources_pool_status/app/yarn_app_heuristic.php?app_id=application_1706529313096_13134832
Spark UI : http://application.tdw.oa.com:8080/proxy/application_1706529313096_13134832


In [2]:
from pyspark.sql import SparkSession
import os
import pandas as pd
import numpy as np
import pickle
from commutil import tdw_util
from IPython.display import Image
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from scipy.stats import randint
from sklearn.ensemble import GradientBoostingRegressor
from causalml.inference.meta import BaseXRegressor, BaseSRegressor

In [3]:
##读取tdw里的分区表
from pytoolkit import TDWSQLProvider
provider = TDWSQLProvider(session, group='tl', db='wxg_mmbiz_dw')
# priParts的分区填写要加上p_, 比如 priParts=['p_20231124']
provider.table('dwmid_daily_wxapp_drama_message_uin_feature_at', priParts=['p_20240130']).createOrReplaceTempView('dwmid_daily_wxapp_drama_message_uin_feature_at')
sql_str='select * from dwmid_daily_wxapp_drama_message_uin_feature_at '
df_raw=session.sql(sql_str)
df_raw_0=df_raw.toPandas()

In [4]:
provider.table('dwd_daily_wxapp_short_moive_uin_msg_optim_feature_predict_at', priParts=['p_20240130']).createOrReplaceTempView('dwd_daily_wxapp_short_moive_uin_msg_optim_feature_predict_at')
sql_str='select * from dwd_daily_wxapp_short_moive_uin_msg_optim_feature_predict_at '
df_raw_1=session.sql(sql_str)
df_raw_1=df_raw_1.toPandas()
df_raw_1['treatment'] = df_raw_1['treatment'].fillna(1)

In [5]:
#删除部分和treatment直接相关的协变量
df_raw_0= df_raw_0.drop(columns=['exp_pv', 'clk_pv','drama_msg_send_num_7d','drama_msg_clk_num_7d','drama_msg_exp_num_7d'])
df_raw_1= df_raw_1.drop(columns=['exp_pv', 'clk_pv','drama_msg_send_num_7d','drama_msg_clk_num_7d','drama_msg_exp_num_7d','last_tonow'])
df_raw_1.dtypes

ds                             object
uin                             int64
treatment                       int64
staytime_long                   int64
pay_amount_1d                   int64
pay_amount_30d                  int64
pv                              int64
history_pv                      int64
drama_num                       int64
history_drama_num               int64
wxapp_num                       int64
history_wxapp_num               int64
first_tonow                   float64
city_level                     object
gender                         object
age                             int64
active_level                   object
pay_amount_level               object
ser_notice_exp_7d               int64
ser_notice_clk_7d               int64
long_ser_notice_exp_7d          int64
long_ser_notice_clk_7d          int64
subscribe_tmpl_cnt_7d           int64
subscribe_tmpl_cnt_history      int64
cancel_tmpl_cnt_7d              int64
subscribe_appcnt                int64
is_cancel   

In [6]:
##数据预处理

# 填充包含空值的行
# 对于object类型的列，用众数填充；对于非object类型的列，用均值填充
df_fill_0 = df_raw_0.apply(lambda x: x.fillna(x.mode()[0]) if x.dtype == 'object' else x.fillna(x.mean()))
df_raw_1.loc[df_raw_1['pay_amount_level'] == '0', 'pay_amount_level'] = '未知'
df_raw_1.loc[df_raw_1['active_level'] == '0', 'active_level'] = '0-无效'
df_raw_1.loc[df_raw_1['city_level'] == '0', 'city_level'] = '未知'
df_fill_1 = df_raw_1.apply(lambda x: x.fillna(x.mode()[0]) if x.dtype == 'object' else x.fillna(x.mean()))


# 独热编码将离散型特征转换为数值类型
df_fill_encoded_0= pd.get_dummies(df_fill_0, columns=['gender','pay_amount_level','active_level','city_level'])
df_fill_encoded_1= pd.get_dummies(df_fill_1, columns=['gender','pay_amount_level','active_level','city_level'])
df_fill_encoded_0

,ds,uin,treatment,staytime_long,pay_amount_1d,pay_amount_30d,pv,history_pv,drama_num,history_drama_num,...,city_level_1-一线城市,city_level_2-新一线城市,city_level_3-二线城市,city_level_4-三线城市,city_level_5-四线城市,city_level_6-五线城市,city_level_7-五线以下城市,city_level_8-港澳台,city_level_9-国外其他,city_level_未知
0,20240130,101940,1,0,0,0,0,14296,0,275,...,0,0,0,1,0,0,0,0,0,0
1,20240130,877585,0,2691,0,0,0,76,0,3,...,0,1,0,0,0,0,0,0,0,0
2,20240130,1027807,1,0,0,0,0,6,0,2,...,0,0,0,1,0,0,0,0,0,0
3,20240130,1062782,0,9713,0,0,0,559,0,28,...,0,0,0,0,1,0,0,0,0,0
4,20240130,1063036,1,0,0,0,0,130,0,4,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255814,20240130,4162330531,1,0,0,0,0,61,0,2,...,0,0,0,1,0,0,0,0,0,0
255815,20240130,4162572205,1,3914,0,0,0,193,0,13,...,0,0,0,0,0,1,0,0,0,0
255816,20240130,4162766030,1,38521,0,12679,0,2139,0,51,...,0,0,0,0,0,0,0,0,0,1
255817,20240130,4162801878,1,0,0,0,0,50,0,3,...,0,0,0,1,0,0,0,0,0,0


In [7]:
df_fill_encoded_1

,ds,uin,treatment,staytime_long,pay_amount_1d,pay_amount_30d,pv,history_pv,drama_num,history_drama_num,...,city_level_1-一线城市,city_level_2-新一线城市,city_level_3-二线城市,city_level_4-三线城市,city_level_5-四线城市,city_level_6-五线城市,city_level_7-五线以下城市,city_level_8-港澳台,city_level_9-国外其他,city_level_未知
0,20240130,257716619,1,10789,0,0,0,506,0,11,...,0,0,0,0,1,0,0,0,0,0
1,20240130,273865665,1,2146,0,0,0,94,0,4,...,0,0,0,0,1,0,0,0,0,0
2,20240130,326205754,1,0,0,0,457,1440,7,18,...,0,0,0,0,0,0,0,0,0,1
3,20240130,329052030,1,151438,0,37600,48,17894,2,382,...,0,1,0,0,0,0,0,0,0,0
4,20240130,600059495,1,0,0,0,0,404,0,35,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85744,20240130,3364020376,1,128149,0,0,0,3169,0,71,...,0,1,0,0,0,0,0,0,0,0
85745,20240130,3407726536,1,39,0,0,0,324,0,14,...,0,0,1,0,0,0,0,0,0,0
85746,20240130,3484327855,1,0,0,0,0,24,0,1,...,0,0,0,0,0,0,0,0,0,1
85747,20240130,4073697722,1,2861,0,0,0,79,0,4,...,0,1,0,0,0,0,0,0,0,0


In [8]:
df_fill_encoded_0.groupby('treatment')['pv'].mean()

treatment
0    26.631467
1    26.855647
Name: pv, dtype: float64

In [9]:

# 分离treatment为0和1的数据
df_treatment_0 = df_raw_0[df_raw_0['treatment'] == 0]
df_treatment_1 = df_raw_0[df_raw_0['treatment'] == 1]

# 计算treatment为0和1的数量
num_treatment_0 = len(df_treatment_0)
num_treatment_1 = len(df_treatment_1)

# 对数量较多的类别进行下采样
if num_treatment_0 > num_treatment_1:
    df_treatment_0 = df_treatment_0.sample(n=num_treatment_1, random_state=40)
else:
    df_treatment_1 = df_treatment_1.sample(n=num_treatment_0, random_state=40)

# 合并下采样后的数据
df_balanced = pd.concat([df_treatment_0, df_treatment_1], axis=0)

# 打乱数据顺序
df_raw_balance= df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
df_raw_balance


,ds,uin,treatment,staytime_long,pay_amount_1d,pay_amount_30d,pv,history_pv,drama_num,history_drama_num,...,pay_amount_level,ser_notice_exp_7d,ser_notice_clk_7d,long_ser_notice_exp_7d,long_ser_notice_clk_7d,subscribe_tmpl_cnt_7d,subscribe_tmpl_cnt_history,cancel_tmpl_cnt_7d,subscribe_appcnt,is_cancel
0,20240130,2669455810,1,0,0,0,158,8234,6,209,...,None,3,0,0,0,0,1,0,1,0
1,20240130,3438852689,1,706,0,0,0,41,0,2,...,未知,11,0,0,0,0,1,0,1,0
2,20240130,410150031,1,0,0,0,0,0,0,0,...,None,0,0,0,0,0,0,0,3,0
3,20240130,442690312,1,172528,990,12480,306,12076,3,300,...,3-高额交易,0,0,0,0,0,1,0,1,0
4,20240130,632374201,1,119934,0,5970,30,5094,1,60,...,2-中额交易,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170525,20240130,1751560427,1,31778,590,1860,293,4543,1,99,...,1-低额交易,0,0,0,0,0,4,0,4,0
170526,20240130,45684306,1,72504,0,100,64,4041,1,91,...,1-低额交易,0,0,0,0,0,2,0,2,0
170527,20240130,2584981106,1,3899,0,0,0,320,0,14,...,未知,3,0,0,0,0,1,0,1,0
170528,20240130,1205216882,1,0,0,0,0,3765,0,119,...,None,0,0,0,0,0,3,0,3,0


In [10]:
# 将DataFrame分为训练集和测试集
df_fill_encoded_0_filtered = df_fill_encoded_0[df_fill_encoded_0['treatment'] == 0]
df_test= df_fill_encoded_0_filtered.append(df_fill_encoded_1)
df_train=df_fill_encoded_0
column_names = df_fill_encoded_0.columns.tolist()
#干预变量
t='treatment'
# 结果变量
label_tag='pv'
# 混淆变量
full_features=[col for col in column_names if col not in [t, label_tag,'uin','ds']]
full_features_sl=[col for col in column_names if col not in [label_tag,'uin','ds']]

In [11]:
df_train

,ds,uin,treatment,staytime_long,pay_amount_1d,pay_amount_30d,pv,history_pv,drama_num,history_drama_num,...,city_level_1-一线城市,city_level_2-新一线城市,city_level_3-二线城市,city_level_4-三线城市,city_level_5-四线城市,city_level_6-五线城市,city_level_7-五线以下城市,city_level_8-港澳台,city_level_9-国外其他,city_level_未知
0,20240130,101940,1,0,0,0,0,14296,0,275,...,0,0,0,1,0,0,0,0,0,0
1,20240130,877585,0,2691,0,0,0,76,0,3,...,0,1,0,0,0,0,0,0,0,0
2,20240130,1027807,1,0,0,0,0,6,0,2,...,0,0,0,1,0,0,0,0,0,0
3,20240130,1062782,0,9713,0,0,0,559,0,28,...,0,0,0,0,1,0,0,0,0,0
4,20240130,1063036,1,0,0,0,0,130,0,4,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255814,20240130,4162330531,1,0,0,0,0,61,0,2,...,0,0,0,1,0,0,0,0,0,0
255815,20240130,4162572205,1,3914,0,0,0,193,0,13,...,0,0,0,0,0,1,0,0,0,0
255816,20240130,4162766030,1,38521,0,12679,0,2139,0,51,...,0,0,0,0,0,0,0,0,0,1
255817,20240130,4162801878,1,0,0,0,0,50,0,3,...,0,0,0,1,0,0,0,0,0,0


In [12]:
##评估函数 uplift柱状图
def uplift_by_percentile(y_true, uplift, treatment):
    y_true_temp, uplift_temp, treatment_temp = np.array(y_true), np.array(uplift), np.array(treatment)

    order = np.argsort(uplift_temp, kind='mergesort')[::-1]
    trmnt_flag = 1
    bins = 10
    
    # 对样本按照uplift值划分分位数
    y_true_bin = np.array_split(y_true_temp[order], bins)
    trmnt_bin = np.array_split(treatment_temp[order], bins)
    
    n_trmnt = np.array([len(y[trmnt == trmnt_flag]) for y, trmnt in zip(y_true_bin, trmnt_bin)]) #treatment组样本数
    n_ctrl = np.array([len(y[trmnt != trmnt_flag]) for y, trmnt in zip(y_true_bin, trmnt_bin)])  #control组样本数
    
    trmnt_avg = np.array([np.mean(y[trmnt == trmnt_flag]) for y, trmnt in zip(y_true_bin, trmnt_bin)]) #treatment组y_true均值
    ctrl_avg = np.array([np.mean(y[trmnt != trmnt_flag]) for y, trmnt in zip(y_true_bin, trmnt_bin)]) #control组y_true均值
    
    uplift_score = trmnt_avg - ctrl_avg   #相减得到uplift score
    
    percentiles = [round(p * 100 / bins) for p in range(1, bins + 1)]
    df = pd.DataFrame({'percentile': percentiles,
                       'n_treatment': n_trmnt,
                       'n_control': n_ctrl,
                       'treatment_avg': trmnt_avg,
                       'control_avg': ctrl_avg,
                       'uplift': uplift_score})

    df = df.set_index('percentile', drop=True, inplace=False) \
           .astype({'n_treatment': 'int32', 'n_control': 'int32'})
    
    return df


def my_plot_uplift_by_percentile(y_true, uplift, treatment):
    df = uplift_by_percentile(y_true, uplift, treatment)
    bins = 10
    percentiles = [round(p * 100 / bins) for p in range(1, bins + 1)]
    delta = percentiles[0]
    treatment_avg = df['treatment_avg']
    control_avg = df['control_avg']
    uplift = df['uplift']
    
    fig, axes = plt.subplots(ncols=1, nrows=2, figsize=(8, 6), sharex=True, sharey=True, facecolor='white')
    #fig.text(0.04, 0.5, 'Uplift = treatment response rate - control response rate', va='center', ha='center', rotation='vertical')

    axes[1].bar(np.array(percentiles) - delta/6, treatment_avg, delta/3, color='forestgreen', label='treatment')
    axes[1].bar(np.array(percentiles) + delta/6, control_avg, delta/3,color='orange', label='control')
    axes[0].bar(np.array(percentiles), uplift, delta/1.5, color='red', label='uplift')

    axes[0].legend(loc='upper right')
    axes[0].tick_params(axis='x', bottom=False)
    axes[0].axhline(y=0, color='black', linewidth=1)
    axes[0].set_title(f'Uplift by percentile')

    axes[1].set_xticks(percentiles)

    axes[1].legend(loc='upper right',prop = {'size':9})
    axes[1].axhline(y=0, color='black', linewidth=1)
    axes[1].set_xlabel('Percentile')
    axes[1].set_title('Response rate by percentile')

    plt.show()

In [13]:
#增益曲线
#AUQC，AUUC
from causalml.metrics import plot_qini, plot_gain
def get_auqc_auuc(label,pred_uplift,treatment):
    # Area Under Qini Curve
    qini_coef = qini_auc_score(label,pred_uplift,treatment)
    # Area Under Uplift Curve
    uplift_auc = uplift_auc_score(label,pred_uplift,treatment)
    print("AUQC: ",qini_coef)
    print("AUUC: ",uplift_auc)
    return qini_coef,uplift_auc

# causalml

In [ ]:
import multiprocessing as mp
from collections import defaultdict

np.random.seed(42)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

import causalml
from causalml.metrics import plot_gain, plot_qini, qini_score
from causalml.dataset import synthetic_data
from causalml.inference.tree import plot_dist_tree_leaves_values, get_tree_leaves_mask
from causalml.inference.meta import BaseSRegressor, BaseXRegressor, BaseTRegressor, BaseDRRegressor
from causalml.inference.tree import CausalRandomForestRegressor
from causalml.inference.tree import CausalTreeRegressor
from causalml.inference.tree.plot import plot_causal_tree
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from causalml.inference.meta import XGBTRegressor
from causalml.inference.meta import BaseSLearner, BaseTLearner, BaseRLearner, BaseXLearner
from causalml.inference.tree import UpliftRandomForestClassifier
from causalml.dataset import *
from causalml.metrics import *

import matplotlib.pyplot as plt
import seaborn as sns

%config InlineBackend.figure_format = 'retina'

## causal tree

In [ ]:
##因果树
ctrees = {
    'ctree_mse': {
        'params':
        dict(criterion='standard_mse',
             control_name=0,
             min_impurity_decrease=0,
             min_samples_leaf=400,
             groups_penalty=0.,
             groups_cnt=True),
    },
    'ctree_cmse': {
        'params':
        dict(
            criterion='causal_mse',
            control_name=0,
            min_samples_leaf=400,
            groups_penalty=0.,
            groups_cnt=True,
            max_depth=6
        ),
    },
    'ctree_cmse_p=0.1': {
        'params':
        dict(
            criterion='causal_mse',
            control_name=0,
            min_samples_leaf=400,
            groups_penalty=0.1,
            groups_cnt=True,
            max_depth=6
        ),
    },
    'ctree_cmse_p=0.25': {
        'params':
        dict(
            criterion='causal_mse',
            control_name=0,
            min_samples_leaf=400,
            groups_penalty=0.25,
            groups_cnt=True,
            max_depth=6
        ),
    },
    'ctree_cmse_p=0.5': {
        'params':
        dict(
            criterion='causal_mse',
            control_name=0,
            min_samples_leaf=400,
            groups_penalty=0.5,
            groups_cnt=True,
            max_depth=6
        ),
    },
    'ctree_ttest': {
        'params':
        dict(criterion='t_test',
             control_name=0,
             min_samples_leaf=400,
             groups_penalty=0.,
             groups_cnt=True,
             max_depth=6
            ),
    },
}

In [ ]:
# Model treatment effect
def tree_model_fit(tree_models,ctrees,df_train,df_test,full_features,t,label_tag):
    '''
        Fit the models in dict and return the result
        Args:
        ----
        tree_models : a dict to save the models
        ctrees: a dict of models name and model params
        df_train: train set
        df_test: test set
        df_raw_0: all of the dataset
        full_features: a list of all the covariates
        t: treatment
        label_tag: target variable
    '''
    for ctree_name, ctree_info in ctrees.items():
        print(f"Fitting: {ctree_name}")
        ctree = CausalTreeRegressor(**ctree_info['params'])
        ctree.fit(X=df_train[full_features].values,
                  treatment=df_train[t].values,
                  y=df_train[label_tag].values)
        tree_models[ctree_name] = ctree

        ctrees[ctree_name].update({'model': ctree})
        df_train[ctree_name] = ctree.predict(df_train[full_features].values)
        df_test[ctree_name] = ctree.predict(df_test[full_features].values)

    #return [df_test,df_raw_0]

In [ ]:
tree_models=dict()
tree_model_fit(tree_models,ctrees,df_train,df_test,full_features,t,label_tag)

In [ ]:
for ctree_name, ctree_info in ctrees.items():
    if ctree_name=='ctree_mse':
        plt.figure(figsize=(20,20))
        plot_causal_tree(ctree_info['model'], 
                         feature_names = full_features,
                         filled=True,
                         impurity=True,
                         proportion=False,
                  )
        plt.title(ctree_name)
        plt.savefig(f'{ctree_name}.svg') 

In [ ]:
#查看每棵树所有叶子结点上uplift预测值的分布
for ctree_name, ctree_info in ctrees.items():
    plot_dist_tree_leaves_values(ctree_info['model'], 
                                 figsize=(3,3),
                                 title=f'Tree({ctree_name}) leaves values distribution')

## causal forest

In [ ]:
##因果森林
cforests = {
    'cforest_mse': {
        'params':
        dict(criterion='standard_mse',
             control_name=0,
             min_impurity_decrease=0,
             min_samples_leaf=400,
             groups_penalty=0.,
             groups_cnt=True),
    },
    'cforest_cmse': {
        'params':
        dict(
            criterion='causal_mse',
            control_name=0,
            min_samples_leaf=400,
            groups_penalty=0.,
            groups_cnt=True
        ),
    },
    'cforest_cmse_p=0.2': {
        'params':
        dict(
            criterion='causal_mse',
            control_name=0,
            min_samples_leaf=400,
            groups_penalty=0.2,
            groups_cnt=True
        ),
    },
    'cforest_cmse_p=0.2_md=5': {
        'params':
        dict(
            criterion='causal_mse',
            control_name=0,
            max_depth=5,
            min_samples_leaf=400,
            groups_penalty=0.2,
            groups_cnt=True
        ),
    },
    
    'cforest_cmse_p=0.5': {
        'params':
        dict(
            criterion='causal_mse',
            control_name=0,
            min_samples_leaf=400,
            groups_penalty=0.5,
            groups_cnt=True,
        ),
    },
    'cforest_cmse_p=0.5_md=5': {
        'params':
        dict(
            criterion='causal_mse',
            control_name=0,
            max_depth=5,
            min_samples_leaf=400,
            groups_penalty=0.5,
            groups_cnt=True,
        ),
    },
    'cforest_ttest': {
        'params':
        dict(criterion='t_test',
             control_name=0,
             min_samples_leaf=400,
             groups_penalty=0.,
             groups_cnt=True),
    },
}

In [ ]:
def forest_model_fit(forest_models,cforests,df_train,df_test,full_features,t,label_tag):
    '''
        Fit the models in dict and add the uplift result to df
        Args:
        ----
        forest_models : a dict to save the models
        cforests: a dict of models name and model params
        df_train: train set
        df_test: test set
        df_raw_0: all of the dataset
        full_features: a list of all the covariates
        t: treatment
        label_tag: target variable
    '''   
    for cforest_name, cforest_info in cforests.items():
        print(f"Fitting: {cforest_name}")
        cforest = CausalRandomForestRegressor(**cforest_info['params'])
        cforest.fit(X=df_train[full_features].values,
                  treatment=df_train[t].values,
                  y=df_train[label_tag].values)
        # save the model in a dict
        forest_models[cforest_name] = cforest

        cforests[cforest_name].update({'model': cforest})
        df_train[cforest_name] = cforest.predict(df_train[full_features].values)
        df_test[cforest_name] = cforest.predict(df_test[full_features].values)
        

In [ ]:
### Model treatment effect
##a dict to save the model
forest_models=dict()
forest_model_fit(forest_models,cforests,df_train,df_test,full_features,t,label_tag)

In [ ]:
'''
forest_models=dict()
for cforest_name, cforest_info in cforests.items():
    print(f"Fitting: {cforest_name}")
    cforest = CausalRandomForestRegressor(**cforest_info['params'])
    cforest.fit(X=df_train[full_features].values,
              treatment=df_train[t].values,
              y=df_train[label_tag].values)
    # save the model in a dict
    forest_models[cforest_name] = cforest
    
    cforests[cforest_name].update({'model': cforest})
    df_test[cforest_name] = cforest.predict(df_test[full_features].values)
    df_raw_0[cforest_name] = cforest.predict(df_raw_0[full_features].values)
'''

In [ ]:
##训练集上的qini
# 获取两个字典的键名
result = list(ctrees.keys()) + list(cforests.keys())
# 添加 t和 label_tag到列表中
result.extend([t, label_tag])

df_qini = qini_score(df_train[result],
           outcome_col=label_tag,
           treatment_col=t)

df_qini.sort_values(ascending=False)


In [ ]:
#测试数据集的qini
df_qini_all = qini_score(df_test[result],
           outcome_col=label_tag,
           treatment_col=t)

df_qini_all.sort_values(ascending=False)

## Meta Learner

In [27]:
#尝试过meta learner的效果不佳，远不如树模型和森林模型，故为减少运行时长先不考虑该类model

X_train = df_train[full_features].values
X_test = df_test[full_features].values
#X = df_raw_0[full_features].values



# learner - DecisionTreeRegressor
# treatment learner - LinearRegression()

learner_x = BaseXRegressor(learner=GradientBoostingRegressor(), 
                           treatment_effect_learner=LinearRegression())
learner_s = BaseSRegressor(learner=GradientBoostingRegressor())
learner_t = BaseTRegressor(learner=GradientBoostingRegressor(), 
                           treatment_learner=LinearRegression())
learner_dr = BaseDRRegressor(learner=GradientBoostingRegressor(), 
                             treatment_effect_learner=LinearRegression())

learner_x.fit(X=X_train, treatment=df_train[t].values, y=df_train[label_tag].values)
learner_s.fit(X=X_train, treatment=df_train[t].values, y=df_train[label_tag].values)
learner_t.fit(X=X_train, treatment=df_train[t].values, y=df_train[label_tag].values)
learner_dr.fit(X=X_train, treatment=df_train[t].values, y=df_train[label_tag].values)

df_test['learner_x_ite'] = learner_x.predict(X_test)
df_test['learner_s_ite'] = learner_s.predict(X_test)
df_test['learner_t_ite'] = learner_t.predict(X_test)
df_test['learner_dr_ite'] = learner_dr.predict(X_test)

#df_raw_0['learner_dr_ite'] = learner_dr.predict(X)
#df_raw_0['learner_x_ite']= learner_x.predict(X)
#df_raw_0['learner_s_ite'] = learner_s.predict(X)
#df_raw_0['learner_t_ite'] = learner_t.predict(X)


In [ ]:
#同上

cate_ctrees = [info['model'].predict(X) for _, info in ctrees.items()]
cate_cforests = [info['model'].predict(X) for _, info in cforests.items()]

model_cate = [
    *cate_ctrees,
    *cate_cforests,
    df_raw_0['learner_x_ite'], df_raw_0['learner_s_ite'], df_raw_0['learner_t_ite'], df_raw_0['learner_dr_ite']
]

model_names = [
    *list(ctrees), *list(cforests),
    'X Learner', 'S Learner', 'T Learner', 'DR Learner']


# 特征选择

In [ ]:
#计算各模型的qini系数，并根据qini系数选出排序前6的model
#result.extend(['learner_x_ite','learner_s_ite','learner_t_ite','learner_dr_ite'])
df_qini = qini_score(df_train[result],
           outcome_col=label_tag,
           treatment_col=t)
df_qini.sort_values(ascending=False)

In [ ]:
# 取排序后的前六个值
top_six = df_qini.sort_values(ascending=False).head(6)
result_top6 = top_six.index.tolist()
result_top6.extend([t,label_tag])
result_top6

In [ ]:
##特征重要性

# 创建一个新的DataFrame，包含特征名和对应的重要性
df_importances = pd.DataFrame({
    'feature': full_features, 
    'importance': forest_models[result_top6[0]].feature_importances_
})

# 按照重要性降序排列
df_importances = df_importances.sort_values('importance', ascending=False)

# 绘制柱状图
fig, ax = plt.subplots(figsize=(12, 4))  # 设置图形大小
df_importances.plot.bar(x='feature', y='importance', ax=ax)
ax.set_title("Causal Forest feature importances")
ax.set_ylabel("Mean decrease in impurity")
ax.set_xticklabels(df_importances['feature'], rotation='vertical')
plt.show()

In [ ]:
# 按照重要性降序排列并选出前15个
top15 = df_importances.nlargest(15, 'importance')

# 将这15个特征的名字转换为list
top15_features = top15['feature'].tolist()
top15_features

In [ ]:
# 特征选择后再拟合因果树
'''
for ctree_name, ctree_info in ctrees.items():
    print(f"Fitting: {ctree_name}")
    ctree = CausalTreeRegressor(**ctree_info['params'])
    ctree.fit(X=df_train[top15_features].values,
              treatment=df_train[t].values,
              y=df_train[label_tag].values)
    #tree_models[ctree_name] = ctree
    
    ctrees[ctree_name].update({'model': ctree})
    df_test[ctree_name] = ctree.predict(df_test[top15_features].values)
    df_raw_0[ctree_name] = ctree.predict(df_raw_0[top15_features].values)
'''

In [ ]:
# 特征选择后再拟合因果森林
forest_models_new=dict()
forest_model_fit(forest_models_new,cforests,df_train,df_test,top15_features,t,label_tag)

In [ ]:
#特征选择后再拟合因果森林
'''
for cforest_name, cforest_info in cforests.items():
    print(f"Fitting: {cforest_name}")
    cforest = CausalRandomForestRegressor(**cforest_info['params'])
    cforest.fit(X=df_train[top15_features].values,
              treatment=df_train[t].values,
              y=df_train[label_tag].values)
    # save the model in a dict
    #forest_models[cforest_name] = cforest
    
    cforests[cforest_name].update({'model': cforest})
    df_test[cforest_name] = cforest.predict(df_test[top15_features].values)
    df_raw_0[cforest_name] = cforest.predict(df_raw_0[top15_features].values)
'''

In [ ]:
# 特征选择后再拟合因果树
tree_models_new=dict()
tree_model_fit(tree_models_new,ctrees,df_train,df_test,top15_features,t,label_tag)

In [ ]:

X_train = df_train[top15_features].values
X_test = df_test[top15_features].values
#X = df_raw_0[top20_features].values

# learner - DecisionTreeRegressor
# treatment learner - LinearRegression()

learner_x = BaseXRegressor(learner=GradientBoostingRegressor(), 
                           treatment_effect_learner=LinearRegression())
learner_s = BaseSRegressor(learner=GradientBoostingRegressor())
learner_t = BaseTRegressor(learner=GradientBoostingRegressor(), 
                           treatment_learner=LinearRegression())
learner_dr = BaseDRRegressor(learner=GradientBoostingRegressor(), 
                             treatment_effect_learner=LinearRegression())

learner_x.fit(X=X_train, treatment=df_train[t].values, y=df_train[label_tag].values)
learner_s.fit(X=X_train, treatment=df_train[t].values, y=df_train[label_tag].values)
learner_t.fit(X=X_train, treatment=df_train[t].values, y=df_train[label_tag].values)
learner_dr.fit(X=X_train, treatment=df_train[t].values, y=df_train[label_tag].values)

df_test['learner_x_ite'] = learner_x.predict(X_test)
df_test['learner_s_ite'] = learner_s.predict(X_test)
df_test['learner_t_ite'] = learner_t.predict(X_test)
df_test['learner_dr_ite'] = learner_dr.predict(X_test)

#df_raw_0['learner_dr_ite'] = learner_dr.predict(X)
#df_raw_0['learner_x_ite']= learner_x.predict(X)
#df_raw_0['learner_s_ite'] = learner_s.predict(X)
#df_raw_0['learner_t_ite'] = learner_t.predict(X)


In [42]:
df_train['learner_dr_ite'] = learner_dr.predict(X_train)
df_train['learner_x_ite']= learner_x.predict(X_train)
df_train['learner_s_ite'] = learner_s.predict(X_train)
df_train['learner_t_ite'] = learner_t.predict(X_train)

# 离线评估

In [ ]:
##整个数据集的qini
#result=result+['learner_x_ite','learner_s_ite','learner_t_ite','learner_dr_ite']
df_qini_new = qini_score(df_test[result],
           outcome_col=label_tag,
           treatment_col=t)

df_qini_new.sort_values(ascending=False)

In [ ]:
#选择效果最好的前6个模型
top_six = df_qini_new.sort_values(ascending=False).head(6)
result_top6 = top_six.index.tolist()
result_top6.extend([t,label_tag])
result_top6

## Gain曲线

In [ ]:
#测试集上的gain曲线
plot_gain(df_test[result_top6], 
          outcome_col=label_tag, 
          treatment_col=t,
          n = df_test.shape[0]
         )

In [ ]:

plot_gain(df_train[list(cforests.keys())+[t,label_tag]],
          outcome_col=label_tag,
          treatment_col=t,
          n = df_raw_0.shape[0]
         )

In [ ]:
#全部数据上的gain曲线
plot_gain(df_train[result_top6],
          outcome_col=label_tag,
          treatment_col=t,
          n = df_raw_0.shape[0]
         )

## Qini曲线

In [ ]:
plot_qini(df_test[result_top6],
          outcome_col=label_tag,
          treatment_col=t
         )

In [ ]:
plot_qini(df_train[result_top6],
          outcome_col=label_tag,
          treatment_col=t,
          figsize=(8,8)
         )

## Uplift bar

In [ ]:
#若uplift by percentile图的趋势为单调递减的柱状图则表示模型是合理的
my_plot_uplift_by_percentile(df_train[label_tag], df_train[result_top6[0]], df_train['treatment'])

# uplift阈值选定&保存结果

In [ ]:
sorted_df = df_train.sort_values(by=result_top6[0], ascending=False).reset_index()

In [ ]:
# 初始化gain_cum列
sorted_df['gain_cum'] = 0

# 计算每一行的gain_cum值
for index, row in sorted_df.iterrows():
    treatment_rows = sorted_df.loc[:index, 'treatment'] == 1
    control_rows = sorted_df.loc[:index, 'treatment'] == 0
    
    treatment_indices = treatment_rows.to_numpy().nonzero()[0]
    control_indices = control_rows.to_numpy().nonzero()[0]

    treatment_pv_sum = sorted_df.loc[treatment_indices, 'pv'].sum()
    control_pv_sum = sorted_df.loc[control_indices, 'pv'].sum()

    treatment_count = treatment_rows.sum()
    control_count = control_rows.sum()
    
    #计算累积增益gain
    if treatment_count > 0 and control_count > 0:
        gain_cum = ((treatment_pv_sum / treatment_count) - (control_pv_sum / control_count)) * (index + 1)
        sorted_df.at[index, 'gain_cum'] = gain_cum

print(sorted_df.head())

In [ ]:
max_gain_cum_index = sorted_df['gain_cum'].idxmax()
max_gain_cum_value = sorted_df['gain_cum'].max()
max_gain_cum_index

In [ ]:
sorted_df.loc[max_gain_cum_index, result_top6[0]]

In [ ]:
uplift_thereshold=sorted_df.loc[max_gain_cum_index, result_top6[0]] #阈值
    
df_test['is_high_uplift'] = 0
    
df_test.loc[df_test[result_top6[0]] >= uplift_thereshold, 'is_high_uplift'] = 1

df_test.groupby('is_high_uplift')[result_top6[0]].mean()   

In [ ]:
df_test.groupby('is_high_uplift').count()

In [ ]:
 ##将结果保存到tdw表中
    #将pandas df转为spark df
    #sorted_df_result=sorted_df[['ds','uin','is_high_uplift']]
sorted_df_result=df_test[df_test['treatment'] == 1][['ds', 'uin', 'is_high_uplift']]
sorted_df_result

In [ ]:
sorted_df['is_high_uplift'] = 0
sorted_df.loc[sorted_df.index <= max_gain_cum_index, 'is_high_uplift'] = 1
sorted_df.head()

In [ ]:
sorted_df.groupby('is_high_uplift')[result_top6[0]].mean()


In [ ]:
#输出训练集的预测结果
df_train_result=df_train[['ds', 'uin','treatment', result_top6[0]]]
df_train_result_sp=session.createDataFrame(df_train_result)
tdw_util.write_df(session,df_train_result_sp,'wxg_mmbiz_dw','dwmid_daily_wxapp_short_moive_message_uin_train_uplift_at',sorted_df['ds'].iloc[0])

In [50]:
##将测试集的预测结果保存到tdw表中
#将pandas df转为spark df
#sorted_df_result=sorted_df[['ds','uin','is_high_uplift']]
sorted_df_result=sorted_df[sorted_df['treatment'] == 1][['ds', 'uin', 'is_high_uplift']]

sorted_df_result_sp=session.createDataFrame(sorted_df_result)
tdw_util.write_df(session,sorted_df_result_sp,'wxg_mmbiz_dw','dwmid_daily_wxapp_short_moive_message_uin_uplift_at',sorted_df['ds'].iloc[0])

